# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

<ipython-input-1-1499ab5789c7>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter
import timm
import torch_optimizer
#from torchvision.models import wide_resnet50_2, resnet18
#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
sys.path.append('/home/hiroki/research/dcase2021_task2/src/functions')
import common as com
import pytorch_modeler as modeler
from pytorch_model import EfficientNet_b1 as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

In [4]:
#model = timm.create_model('efficientnet_b1', pretrained=True)
#3, 320, 320
#name: (channel_multiplier, depth_multiplier, resolution, dropout_rate)
#(1.4, 1.8, 380, 0.4)

In [5]:
#model.blocks[0][0]

## load config and set logger

In [6]:
log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

## Setting

In [7]:
# Setting seed
modeler.set_seed(42)

In [8]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_dir = INPUT_ROOT + "/dev_data"
add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = os.listdir(dev_dir)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/DCASE_experiments/SSL-Efficientnet_Barlow_Twins_mixup_0.4_batchnorm/dcase2021_task2/config.yaml'

In [9]:
torch.cuda.is_available()

True

## make path list

- __train_paths__ ([dict]): Each Machine Type
    - __ToyCar__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) :paths
    - __ToyTrain__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) : paths
    - ...

In [10]:
dev_paths = {}

for machine_type in machine_types:
    # dev train
    dev_train_paths = [f"{dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/train")]
    dev_train_paths = sorted(dev_train_paths)
    
    print(f'Machine {machine_type}', end=',')
    print(f'dev_train_size : {len(dev_train_paths)}', end=', ')
    dev_train_source_paths = [path for path in dev_train_paths if 'source' in path]
    print(f'source_size : {len(dev_train_source_paths)}', end=', ')
    dev_train_target_paths = [path for path in dev_train_paths if 'target' in path]
    print(f'target_size : {len(dev_train_target_paths)}', end=', ')
    # source mode
    dev_train_paths = dev_train_source_paths + dev_train_target_paths
    
    # add_dev train
    add_dev_paths = [f"{add_dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{add_dev_dir}/{machine_type}/train")]
    add_dev_paths = sorted(add_dev_paths)
    print(f'adddev_train_size : {len(dev_train_paths)}', end=', ')
    add_dev_source_paths = [path for path in add_dev_paths if 'source' in path]
    print(f'source_size : {len(add_dev_source_paths)}', end=', ')
    add_dev_target_paths = [path for path in add_dev_paths if 'target' in path]
    print(f'target_size : {len(add_dev_target_paths)}')
    # source mode
    add_dev_paths = add_dev_source_paths + add_dev_target_paths
    print('==============================================================')
    
    # dev_source valid
    dev_source_paths = [f"{dev_dir}/{machine_type}/source_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/source_test")]
    dev_source_paths = sorted(dev_source_paths)
    
    # dev_target valid
    dev_target_paths = [f"{dev_dir}/{machine_type}/target_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/target_test")]
    dev_target_paths = sorted(dev_target_paths)
    
    # bundle
    dev_paths[machine_type] = {}
    dev_paths[machine_type]['train'] = dev_train_paths + add_dev_paths
    dev_paths[machine_type]['valid_source'] = dev_source_paths
    dev_paths[machine_type]['valid_target'] = dev_target_paths

Machine fan,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine gearbox,dev_train_size : 3026, source_size : 3017, target_size : 9, adddev_train_size : 3026, source_size : 3105, target_size : 9
Machine pump,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine slider,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyCar,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyTrain,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine valve,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9


In [11]:
machine_types

['fan', 'gearbox', 'pump', 'slider', 'ToyCar', 'ToyTrain', 'valve']

In [12]:
#############################################################################
# run
#############################################################################
def run(machine_type, dev_paths):
    com.tic()
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(dev_paths, machine_type)
    # parameter setting
    logger.info('FEATURE EXTRACTION')
    net = Model(n_out=config['param']['mel_bins'])
    optimizer = torch_optimizer.RAdam(net.parameters())
    #scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001, steps_per_epoch=len(dataloaders_dict['train']), epochs=config['param']['num_epochs'])
    output_dicts = modeler.train_net(net, dataloaders_dict, writer, optimizer)
    
    out_path = f'{MODEL_DIR}/{machine_type}_model.pkl'
    torch.save(output_dicts['net'], out_path)
    logger.info(f'SAVE SUCCESS : {out_path}')
    
    com.toc()

# run

In [13]:
for machine_type in machine_types[1:]:
    run(machine_type, dev_paths)

2021-08-13 02:02:22,904 - 00_train.py - INFO - TARGET MACHINE_TYPE: gearbox
2021-08-13 02:02:22,905 - 00_train.py - INFO - MAKE DATA_LOADER
2021-08-13 02:02:22,905 - 00_train.py - INFO - FEATURE EXTRACTION


use: cuda:0


100%|██████████| 24/24 [00:49<00:00,  2.04s/it]
2021-08-13 02:03:15,100 - pytorch_modeler.py - INFO - epoch:1/200, tr_loss:29650.291382
100%|██████████| 24/24 [00:24<00:00,  1.01s/it]
2021-08-13 02:03:39,335 - pytorch_modeler.py - INFO - epoch:2/200, tr_loss:28085.447144
100%|██████████| 24/24 [00:24<00:00,  1.02s/it]
2021-08-13 02:04:03,762 - pytorch_modeler.py - INFO - epoch:3/200, tr_loss:24405.552673
100%|██████████| 24/24 [00:24<00:00,  1.04s/it]
2021-08-13 02:04:28,709 - pytorch_modeler.py - INFO - epoch:4/200, tr_loss:21916.316589
100%|██████████| 24/24 [00:24<00:00,  1.01s/it]
2021-08-13 02:04:52,987 - pytorch_modeler.py - INFO - epoch:5/200, tr_loss:21155.662842
100%|██████████| 24/24 [00:24<00:00,  1.01s/it]
2021-08-13 02:05:17,261 - pytorch_modeler.py - INFO - epoch:6/200, tr_loss:20312.274109
100%|██████████| 24/24 [00:24<00:00,  1.01s/it]
2021-08-13 02:05:41,534 - pytorch_modeler.py - INFO - epoch:7/200, tr_loss:19655.261658
100%|██████████| 24/24 [00:24<00:00,  1.01s/it]


100%|██████████| 24/24 [00:23<00:00,  1.00it/s]
2021-08-13 02:27:24,895 - pytorch_modeler.py - INFO - epoch:61/200, tr_loss:6318.385239
100%|██████████| 24/24 [00:23<00:00,  1.01it/s]
2021-08-13 02:27:48,739 - pytorch_modeler.py - INFO - epoch:62/200, tr_loss:6335.214783
100%|██████████| 24/24 [00:23<00:00,  1.00it/s]
2021-08-13 02:28:12,679 - pytorch_modeler.py - INFO - epoch:63/200, tr_loss:6146.905212
100%|██████████| 24/24 [00:24<00:00,  1.02s/it]
2021-08-13 02:28:37,214 - pytorch_modeler.py - INFO - epoch:64/200, tr_loss:6147.480606
100%|██████████| 24/24 [00:23<00:00,  1.01it/s]
2021-08-13 02:29:01,064 - pytorch_modeler.py - INFO - epoch:65/200, tr_loss:6133.137360
100%|██████████| 24/24 [00:23<00:00,  1.01it/s]
2021-08-13 02:29:24,852 - pytorch_modeler.py - INFO - epoch:66/200, tr_loss:6097.150330
100%|██████████| 24/24 [00:23<00:00,  1.01it/s]
2021-08-13 02:29:48,675 - pytorch_modeler.py - INFO - epoch:67/200, tr_loss:5973.490067
100%|██████████| 24/24 [00:23<00:00,  1.00it/s]


100%|██████████| 24/24 [00:23<00:00,  1.00it/s]
2021-08-13 02:51:26,909 - pytorch_modeler.py - INFO - epoch:121/200, tr_loss:4206.881180
100%|██████████| 24/24 [00:23<00:00,  1.01it/s]
2021-08-13 02:51:50,755 - pytorch_modeler.py - INFO - epoch:122/200, tr_loss:4246.892685
100%|██████████| 24/24 [00:23<00:00,  1.00it/s]
2021-08-13 02:52:14,657 - pytorch_modeler.py - INFO - epoch:123/200, tr_loss:4224.963852
100%|██████████| 24/24 [00:24<00:00,  1.02s/it]
2021-08-13 02:52:39,192 - pytorch_modeler.py - INFO - epoch:124/200, tr_loss:4208.790985
100%|██████████| 24/24 [00:23<00:00,  1.00it/s]
2021-08-13 02:53:03,143 - pytorch_modeler.py - INFO - epoch:125/200, tr_loss:4148.094604
100%|██████████| 24/24 [00:23<00:00,  1.01it/s]
2021-08-13 02:53:26,955 - pytorch_modeler.py - INFO - epoch:126/200, tr_loss:4109.629303
100%|██████████| 24/24 [00:23<00:00,  1.00it/s]
2021-08-13 02:53:50,871 - pytorch_modeler.py - INFO - epoch:127/200, tr_loss:4074.861511
100%|██████████| 24/24 [00:23<00:00,  1.0

...

100%|██████████| 24/24 [01:38<00:00,  4.11s/it]
2021-08-13 08:44:23,498 - pytorch_modeler.py - INFO - epoch:1/200, tr_loss:29744.763306
100%|██████████| 24/24 [00:23<00:00,  1.02it/s]
2021-08-13 08:44:47,085 - pytorch_modeler.py - INFO - epoch:2/200, tr_loss:27721.171143
100%|██████████| 24/24 [00:23<00:00,  1.03it/s]
2021-08-13 08:45:10,484 - pytorch_modeler.py - INFO - epoch:3/200, tr_loss:24203.749756
100%|██████████| 24/24 [00:23<00:00,  1.02it/s]
2021-08-13 08:45:34,012 - pytorch_modeler.py - INFO - epoch:4/200, tr_loss:22598.127380
100%|██████████| 24/24 [00:23<00:00,  1.03it/s]
2021-08-13 08:45:57,384 - pytorch_modeler.py - INFO - epoch:5/200, tr_loss:21550.601929
100%|██████████| 24/24 [00:24<00:00,  1.00s/it]
2021-08-13 08:46:21,421 - pytorch_modeler.py - INFO - epoch:6/200, tr_loss:21064.667908
100%|██████████| 24/24 [00:23<00:00,  1.01it/s]
2021-08-13 08:46:45,171 - pytorch_modeler.py - INFO - epoch:7/200, tr_loss:20188.910706
100%|██████████| 24/24 [00:23<00:00,  1.03it/s]


100%|██████████| 24/24 [00:23<00:00,  1.02it/s]
2021-08-13 09:07:59,616 - pytorch_modeler.py - INFO - epoch:61/200, tr_loss:5957.879379
100%|██████████| 24/24 [00:24<00:00,  1.00s/it]
2021-08-13 09:08:23,632 - pytorch_modeler.py - INFO - epoch:62/200, tr_loss:5862.075180
100%|██████████| 24/24 [00:23<00:00,  1.02it/s]
2021-08-13 09:08:47,221 - pytorch_modeler.py - INFO - epoch:63/200, tr_loss:5943.247833
100%|██████████| 24/24 [00:23<00:00,  1.02it/s]
2021-08-13 09:09:10,663 - pytorch_modeler.py - INFO - epoch:64/200, tr_loss:5949.180969
100%|██████████| 24/24 [00:23<00:00,  1.02it/s]
2021-08-13 09:09:34,169 - pytorch_modeler.py - INFO - epoch:65/200, tr_loss:5853.293716
100%|██████████| 24/24 [00:23<00:00,  1.03it/s]
2021-08-13 09:09:57,561 - pytorch_modeler.py - INFO - epoch:66/200, tr_loss:5699.171661
100%|██████████| 24/24 [00:24<00:00,  1.00s/it]
2021-08-13 09:10:21,653 - pytorch_modeler.py - INFO - epoch:67/200, tr_loss:5653.863403
100%|██████████| 24/24 [00:23<00:00,  1.02it/s]


100%|██████████| 24/24 [00:23<00:00,  1.02it/s]
2021-08-13 09:31:35,123 - pytorch_modeler.py - INFO - epoch:121/200, tr_loss:4433.061295
100%|██████████| 24/24 [00:23<00:00,  1.03it/s]
2021-08-13 09:31:58,468 - pytorch_modeler.py - INFO - epoch:122/200, tr_loss:4358.087784
100%|██████████| 24/24 [00:24<00:00,  1.00s/it]
2021-08-13 09:32:22,498 - pytorch_modeler.py - INFO - epoch:123/200, tr_loss:4413.906967
100%|██████████| 24/24 [00:23<00:00,  1.01it/s]
2021-08-13 09:32:46,152 - pytorch_modeler.py - INFO - epoch:124/200, tr_loss:4390.052856
100%|██████████| 24/24 [00:23<00:00,  1.02it/s]
2021-08-13 09:33:09,599 - pytorch_modeler.py - INFO - epoch:125/200, tr_loss:4427.882507
100%|██████████| 24/24 [00:23<00:00,  1.02it/s]
2021-08-13 09:33:33,046 - pytorch_modeler.py - INFO - epoch:126/200, tr_loss:4227.895706
100%|██████████| 24/24 [00:23<00:00,  1.02it/s]
2021-08-13 09:33:56,624 - pytorch_modeler.py - INFO - epoch:127/200, tr_loss:4211.077011
100%|██████████| 24/24 [00:24<00:00,  1.0

100%|██████████| 24/24 [00:23<00:00,  1.02it/s]
2021-08-13 09:55:11,108 - pytorch_modeler.py - INFO - epoch:181/200, tr_loss:3654.146027
100%|██████████| 24/24 [00:23<00:00,  1.03it/s]
2021-08-13 09:55:34,515 - pytorch_modeler.py - INFO - epoch:182/200, tr_loss:3586.626450
100%|██████████| 24/24 [00:23<00:00,  1.02it/s]
2021-08-13 09:55:58,056 - pytorch_modeler.py - INFO - epoch:183/200, tr_loss:3521.812836
100%|██████████| 24/24 [00:23<00:00,  1.01it/s]
2021-08-13 09:56:21,898 - pytorch_modeler.py - INFO - epoch:184/200, tr_loss:3499.548904
100%|██████████| 24/24 [00:23<00:00,  1.02it/s]
2021-08-13 09:56:45,461 - pytorch_modeler.py - INFO - epoch:185/200, tr_loss:3479.558716
100%|██████████| 24/24 [00:23<00:00,  1.02it/s]
2021-08-13 09:57:08,945 - pytorch_modeler.py - INFO - epoch:186/200, tr_loss:3493.351074
100%|██████████| 24/24 [00:23<00:00,  1.03it/s]
2021-08-13 09:57:32,321 - pytorch_modeler.py - INFO - epoch:187/200, tr_loss:3546.561615
100%|██████████| 24/24 [00:23<00:00,  1.0

elapsed time: 4794.694454432 [sec]
